In [ ]:
import boto3
import psycopg2
import pandas as pd

In [ ]:
connection = psycopg2.connect(user="postgres", password="xxxxxxx",
                                  host="xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx.amazonaws.com", port="5432",
                                  database="postgres")
cursor = connection.cursor()


In [ ]:
#creating table passengers, survival, tripinfo
cursor.execute("""CREATE TABLE passengers(
id SERIAL PRIMARY KEY,
name text,
sex text,
age float,
sibsp integer,
parch integer)""")

cursor.execute("""CREATE TABLE survival(
id SERIAL PRIMARY KEY,
survived integer)""")

cursor.execute("""CREATE TABLE tripInfo(
id SERIAL PRIMARY KEY,
pclass integer,
ticket text,
fare float,
cabin text,
embarked text)""")

connection.commit()

In [ ]:
sql = """
SELECT "table_name","column_name", "data_type", "table_schema"
FROM INFORMATION_SCHEMA.COLUMNS
WHERE "table_schema" = 'public'
ORDER BY table_name  
"""
pd.read_sql(sql, con=connection)

In [ ]:
#Read in the data
with open('passengers.csv', 'r') as row:
    next(row)# Skip the header row.
    cursor.copy_from(row, 'passengers', sep=',')
    
with open('survival.csv', 'r') as row:
    next(row) 
    cursor.copy_from(row, 'survival', sep=',')

with open('trip_info.csv', 'r') as row:
    next(row)
    cursor.copy_from(row, 'tripinfo', sep=',')
    
connection.commit()  

In [ ]:
# Create a new Role gastudent
cursor.execute("""CREATE ROLE gastudent WITH LOGIN PASSWORD 'gastudent'""")
# Grant different privileges for a new role
cursor.execute("""GRANT CONNECT ON DATABASE postgres TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON passengers TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON survival TO gastudent""")
cursor.execute("""GRANT SELECT, UPDATE, INSERT ON tripInfo TO gastudent""")

connection.commit()

In [ ]:
connection1 = psycopg2.connect(user="gastudent", password="gastudent",
                                  host="xxxxxxxxxxxxxxxxxxxxxxxxx", port="5432",
                                  database="postgres")
cursor1 = connection1.cursor()


In [ ]:
# Now we can query the data! Lets calculate the surcical rate for each passenger's class
sql = """
SELECT pclass, AVG(survived)
FROM (SELECT t.pclass, s.survived
FROM tripInfo t
INNER JOIN survival s ON t.id = s.id
) AS x
GROUP BY pclass
"""

pd.read_sql(sql, con=connection)